In [ ]:
# USAGE
# python detect_mask_video.py
# importer les bibliothèques nécessaires
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import argparse
import imutils
import time
import cv2
import os
import time
from PIL import Image
from datetime import datetime
from time import strftime 
import imageio
def detect_and_predict_mask(frame, faceNet, maskNet):#construction de notre détecteurs de visages et de masques
	# grab the dimensions of the frame and then construct a blob
	# from it
	(h, w) = frame.shape[:2]##hauteur et largeur de l'image
	blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),
		(104.0, 177.0, 123.0))
#frame: Il s'agit de l'image d'entrée que nous voulons prétraiter avant de la transmettre à notre réseau neuronal profond pour la classification.
#scalefactor: Après avoir effectué une soustraction moyenne, nous pouvons éventuellement mettre à l'échelle nos images par un certain facteur. Cette valeur par défaut est «1.0» (c'est-à-dire pas de mise à l'échelle) mais nous pouvons également fournir une autre valeur. Il est également important de noter que scalefactor doit être 1 / \ sigma car nous multiplions en fait les canaux d'entrée (après soustraction de la moyenne) par scalefactor.
#size: Ici, nous fournissons la taille spatiale attendue par le réseau de neurones convolutifs. Pour la plupart des réseaux de neurones de pointe actuels, il s'agit de 224 × 224, 227 × 227 ou 299 × 299.
#signifie: ce sont nos valeurs moyennes de soustraction. Ils peuvent être un 3-tuple des moyens RVB ou ils peuvent être une valeur unique, auquel cas la valeur fournie est soustraite de chaque canal de l'image. Si vous effectuez une soustraction moyenne, assurez-vous de fournir le 3-tuple dans l'ordre `(R, V, B)`, en particulier lorsque vous utilisez le comportement par défaut de swapRB = True.
#swapRB: OpenCV suppose que les images sont dans l'ordre des canaux BGR; cependant, la valeur «moyenne» suppose que nous utilisons l'ordre RVB. Pour résoudre cet écart, nous pouvons permuter les canaux R et B dans l'image en définissant cette valeur sur «True». Par défaut, OpenCV effectue ce changement de chaîne pour nous.
#La fonction cv2.dnn.blobFromImage renvoie un blob qui est notre image d'entrée après la soustraction moyenne, la normalisation et le changement de canal.
#détection de visages
	faceNet.setInput(blob)
	detections = faceNet.forward()


	faces = []#listes des visages
	locs = []#listes des localisations où se trouvent les visages 
	preds = []#listes des prédictions
	# les détections
	for i in range(0, detections.shape[2]):
		confidence = detections[0, 0, i, 2]


	#filtrer les détections faibles qui seront considérés comme des erreurs
		if confidence > 0.5:
		#calculer les coordonnées des limites de la régions d'intérêt
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")

			# ensure the bounding boxes fall within the dimensions of
			# the frame
			(startX, startY) = (max(0, startX), max(0, startY))
			(endX, endY) = (min(w - 1, endX), min(h - 1, endY))

			# extract the face ROI, convert it from BGR to RGB channel
			# ordering, resize it to 224x224, and preprocess it
			face = frame[startY:endY, startX:endX]#ROI
			face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)#convert to the BGR color space
			face = cv2.resize(face, (224, 224))#redimensionnement
			face = img_to_array(face)#transformation en matrice
			face = preprocess_input(face)#la normalisation des valeurs des pixels en [-1, 1]

			# add the face and bounding boxes to their respective
			# lists
			faces.append(face)
			locs.append((startX, startY, endX, endY))#localisation où setrouve l'image

	# only make a predictions if at least one face was detected
	if len(faces) > 0:
		# for faster inference we'll make batch predictions on *all*
		# faces at the same time rather than one-by-one predictions
		# in the above `for` loop
		faces = np.array(faces, dtype="float32")
		preds = maskNet.predict(faces, batch_size=32)#prédire avec le maskNet model
	return (locs, preds)



# load our serialized face detector model from disk
print("[INFO] loading face detector model...")
prototxtPath = os.path.sep.join(["face_detector", "deploy.prototxt"])
weightsPath = os.path.sep.join(["face_detector",
	"res10_300x300_ssd_iter_140000.caffemodel"])
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)#construction du détecteur de visages

# load the face mask detector model from disk
print("[INFO] loading face mask detector model...")
maskNet = load_model("mask_detector.model")

# Activer la webcam et la considérer comme le flux d'image d'entrée
print("[INFO] starting video stream...")
vs = VideoStream(src=1).start()

nbr_vis=0
avec=0
sans=0

while True:
	frame = vs.read()#pour lire la vidéo
	frame = imutils.resize(frame, width=400)#redimensionnement avec une largeur de 400 pixels

#détections de visages et prédictions avec ou sans masques
	(locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

	for (box, pred) in zip(locs, preds):
	#calculer les coordonnées des limites de la régions d'intérêt
		(startX, startY, endX, endY) = box
		(mask, withoutMask) = pred

        
     
		# déterminer la plus grande probabilité et ajouter du texte et un rectangle au visage détecté
		if mask > withoutMask:
		    now = datetime.now().time()
		    time1=now.strftime("%H:%M:%S")
		    label="Mask"
		    color = (0, 255, 0)
		    avec+=1
		    label2 = "{}: {:.2f}% ".format(label,mask* 100)
		    label3= "{}".format(time1)
		    cv2.putText(frame, label2, (startX, startY - 10),
		    cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)                        
		    cv2.putText(frame, label3, (320,30),                        
		    cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
		    cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
		else:           
		    now = datetime.now().time()
		    time1=now.strftime("%H:%M:%S")
		    label="No mask"
		    color = (0, 0, 255)
		    sans+=1
		    label2 = "{}: {:.2f}% ".format(label,withoutMask* 100)
		    label3= "{}".format(time1)            
		    cv2.putText(frame, label2, (startX, startY - 10),
		    cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
		    cv2.putText(frame, label3, (320, 30),            
		    cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)   
		    cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
		    nom= "sans_masque{}.PNG".format(sans)           
		    cv2.imwrite(nom,frame)
		    time.sleep(5)  #arrêter le programme pendant 5s             
		nbr_vis=avec+sans            
         

		# Mettre un texte et un contour sur les visages détectés qui diffère selon s'il est avec ou sans masque
		cv2.putText(frame, label, (startX, startY - 10),
			cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
		label1="Il y'a eu {} personnes sans masque ".format(sans)
		cv2.putText(frame, label1 , (50,40),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255,255), 2)

	# affichage de l'image de sortie
	cv2.imshow("Frame", frame)
	key = cv2.waitKey(0) & 0xFF

	# pour arrêter le programme cliquer q
	if key == ord("q"):
		break

cv2.destroyAllWindows()
vs.stop()
#comment régler les accents sur python?
#quand je transforme les images en array, je le retransforme en image puis en array les 2 array n'ont pas la même valeur 

[INFO] loading face detector model...
[INFO] loading face mask detector model...
[INFO] starting video stream...
